Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [4]:
import pandas as pd
import numpy as np

#we create the dataframes
df = pd.read_csv("bank_marketing.csv")

#client dataframe
clients_df = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage']]

#campaign dataframe
campaign_df = df[['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome']]

#economics dataframe
economics_df = df[['client_id','cons_price_idx', 'euribor_three_months']]

Creating the clients csv

In [5]:
# WE WORK IN A COPY OF THE CLIENTS DATAFRAME TO STORE IT IN A CSV
clients_df_copy = clients_df.copy()

# We change '.' to '_' in job columnn values
clients_df_copy['job'] = clients_df_copy['job'].str.replace('.', '_')

# We change '.' to '_' and 'unknown' to np.NaN in education column values
clients_df_copy['education'] = clients_df_copy['education'].str.replace('.', '_')
clients_df_copy['education'] = clients_df_copy['education'].replace('unknown', np.NaN)

#Convert to boolean data type: 1 if "yes", otherwise "0" in credit_default column values
map_values = {'yes':True, 'success' : True, 'no' : False, 'failure' : False, 'nonexistent': False}
clients_df_copy['credit_default'] = clients_df_copy['credit_default'].map(map_values).astype(bool)

#Convert to boolean data type: 1 if "yes", otherwise "0" in mortgage column values
map_values_cf = {'yes' : True, 'no' : False, 'unknown' : False}
clients_df_copy['mortgage'] = clients_df_copy['mortgage'].map(map_values_cf).astype(bool)


In [6]:
# We save our dataframe into a csv
clients_df_copy.to_csv('client.csv', index = False)

Creating the campaign csv

In [31]:
# We create a copy of our dirty campaign csv
campaign_df_copy = campaign_df.copy()

# Convert to boolean data type 1 if 'success' otherwise 0
previous_outcome_values = {'success':True, 'failure' : False, 'nonexistent' : False}
campaign_df_copy['previous_outcome'] = campaign_df_copy['previous_outcome'].map(previous_outcome_values)

# Convert to boolean data type True if yes otherwise 0
campaign_outcome_values = {'no' : False, 'yes' : True}
campaign_df_copy['campaign_outcome'] = campaign_df_copy['campaign_outcome'].map(campaign_outcome_values)

# Create from a combination of day, month, and a newly created year column (which should have a value of 2022), Format = "YYYY-MM-DD"
campaign_df_copy['year'] = 2022

# I merge the 2 dataframes in order to have the year, month and day in one dataframe so I can create the date
merge_df = pd.merge(df, campaign_df_copy, left_index=True, right_index=True)

# We change the months to ints
month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}

merge_df['month'] = merge_df['month'].map(month_mapping)
campaign_df_copy['last_contact_date'] = pd.to_datetime(merge_df[['year', 'month', 'day']],format='%Y-%M-%D')

0   2022-05-13
1   2022-05-19
2   2022-05-23
3   2022-05-27
4   2022-05-03
Name: last_contact_date, dtype: datetime64[ns]


In [32]:
# We save our dataframe into a csv file
campaign_df_copy.to_csv('campaign.csv', index=False)

We create the economics csv

In [33]:
economics_df.to_csv('economics.csv', index=False)

In [6]:
for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64
